In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

# Logik

In [ ]:
%run ./Muehle_Utilities.ipynb
import copy

## Spielphasen

Die Funktion `get_player_phase()` prüft in welcher Phase sich der übergebenen Spieler befindet. Hat er noch setzbare Steine, ist also sein Wert in `remaining` größer null, befindet er sich in Phase eins. Ist dies nicht der Fall und auch auf dem Feld befinden sich nur noch drei Steine, so befindet der Spieler sich in der Springphase. Ist dem nicht so, so ist der Spieler in Phase zwei, der Zugphase.
Die Eingabeparameter der Funktion sind ein Spielstatus, sowie ein Spieler in der Form 1 für Spieler eins und 2 für Spieler zwei. Zurückgegeben wird die Spielphase des übergebenen Spielers in dem übergebenen Spiel.

In [ ]:
def get_player_phase(state, player):
    remaining, board = state
    if remaining[player - 1] >= 1: 
        return 1
    elif count_player_pieces(board, player) <= 3:
        return 3
    else:
        return 2

`next_positions()` berechnet in welcher Phase sich der Spieler befindet und dementsprechend, wo er als nächstes klicken kann. Falls der Spieler in Phase 1 (Setzphase) oder Phase 3 (Springphase) ist, sind dies unabhängig vom aktuellen Spieler oder in Phase 3 auch unabhängig vom ausgewählten Stein alle leeren Felder. Für Phase 2 wird die zuvor definierte Funktion `empty_neighbors()` aufgerufen.
Übergeben wird der Funktion `next_positions()` hierfür ein Spielstatus, ein Spieler, und die Position des ausgewählten Steines (ring und cell).

In [ ]:
def next_positions(state, player, ring, cell):
    _, board = state
    phase    = get_player_phase(state, player)
    return empty_positions(board) if phase != 2 else empty_neighbors(board, ring, cell)

## next states

Im Folgenden werden Funktionen implementiert, die je nach Spielphase alle möglichen nächsten Zustände für einen Spieler bei  gegebenem Zustand ausrechnet. Es existieren dementsprechend die Methoden
* next_states_phase_one()
* next_states_phase_two()
* next_states_phase_three()

Alle drei nehmen einen Zustand und einen Spieler und geben eine List möglicher nächster Zustände zurück. Der Ablauf aller drei Methoden ist im Allgemeinen folgendermaßen:
1. Aktuelle Mühlen suchen
2. Steine positionieren/verschieben und entstehende Bretter in einer Liste speichern
3. Für jedes Board überprüfen, ob neue Mühlen dazu gekommen sind und ggf. Steine des Gegners entfernen
4. next_states zusammenbauen und zurückgeben

Für **Schritt 1** wird die Funktion `find_mills()` genutzt. Die Phasen unterscheiden sich im Wesentlichen in **Schritt 2**, weshalb dieser für die einzelnen Phasen gleich genauer betrachtet werden soll. In **Schritt 3** wird die Funktion `generate_final_boards()` aufgerufen. In dieser wird über die zuvor entwickelten Zustände iteriert und für jedes Brett zunächst erneut `find_mills()` und damit zusammen mit den zu Beginn ermittelten Mühlen `count_new_mills()` aufgerufen. Letztere Funktion liefert eine natürliche Zahl, die angibt, wie viele neue Mühlen durch den Wechsel in den Zustand entstehen. Falls das Ergebnis 0 ist, geschieht nichts. Falls die Zahl größer 0 ist, wird pro entstandener Mühle einmal die Funktion `beat_pieces()` aufgerufen, die jeweils einen schlagbaren Stein des Gegners entfernt. In diesem Fall wird das berechnete Board also nochmal manipuliert. In **Schritt 4** wird der Zustand, der neben dem Board auch aus der Variable `remaining` besteht, zusammengesetzt.

In [ ]:
def generate_final_boards(boards_after_placing, player):
    boards = []
    for new_board in boards_after_placing:
        number_new_mills = len(find_new_mills())
        if number_new_mills > 0:
            boards.extend(beat_pieces(new_board, number_new_mills, player))
        else:
            boards.append(new_board)
    return boards

### Phase 1
In Phase 1 wird ein Stein auf ein leeres Feld gesetzt. Die nächsten Zustände entstehen also, in dem über alle leeren Felder iteriert und dort jeweils ein Stein positioniert wird. Dies geschieht als Schritt 2 in `next_states_phase_one()`. Die Besonderheit in der ersten Phase ist, dass in Schritt 4 auch die Variable `remaining` angepasst werden muss, in dem Spieler ein Stein abgezogen wird.

In [ ]:
def next_states_phase_one(state, player):
    remaining, board = state
    
    #mills = find_mills(board, player) # Schritt 1
    
    # Schritt 2
    empty = empty_positions(board)
    boards_after_placing = []
    for ring, cell in empty:
        new_board = copy.deepcopy(board)
        new_board[ring][cell] = player        
        boards_after_placing.append(new_board)
    
    boards = generate_final_boards(boards_after_placing, player) # Schritt 3
    
    remaining = [remaining[0] - 1, remaining[1]] if player == 1 else [remaining[0], remaining[1] - 1] # Schritt 4
    
    return [[remaining, new_board] for new_board in boards]

### Phase 2
Die nächsten Zustände entstehen in der Zugphase durch das Verschieben eines eigenen Steins auf ein benachbartes freies Feld. Im Schritt 2 von `next_states_phase_two()` werden also zunächst alle Spielsteine eines Spielers ermittelt, um anschließend über die iterieren zu können. Für jeden dieser Steine werden dann mit Hilfe von `empty_neighbors()` alle leeren benachbarten Felder ausgerechnet. Pro Stein und freie Nachbar-Position entsteht ein möglicher neuer Zustand indem der Wert der Startposition auf 0 und der Wert der freien Nachbar-Position, also der Zielposition, auf den Wert des Spielers gesetzt wird.

In [ ]:
def next_states_phase_two(state, player):
    remaining, board = state
    
    #mills = find_mills(board, player) # Schritt 1
    
    # Schritt 2
    pieces = player_pieces(board, player)
    boards_after_placing = []
    for ring_start, cell_start in pieces:
        positions = empty_neighbors(board, ring_start, cell_start)
        for ring_goal, cell_goal in positions:
            new_board = copy.deepcopy(board)
            new_board[ring_start][cell_start] = 0    
            new_board[ring_goal][cell_goal] = player        
            boards_after_placing.append(new_board)
    
    boards = generate_final_boards(boards_after_placing, player)  # Schritt 3
    
    return [[remaining, new_board] for new_board in boards] # Schritt 4

### Phase 3
In Phase 3 kann ein beliebiger eigener Stein auf ein beliebiges freies Feld versetzt werden. Dazu wird in Schritt 2 von `next_states_phase_three()` zunächst nach eigenen Steinen (Startfelder) und leeren Positionen (Zielfelder) gesucht. Anschließende entstehen mögliche Spielbretter aus allen möglichen Kombinationen von Start- und Zielfeldern. Das Startfeld erhält den Wert 0, das Zielfeld den Wert des Spielers.

In [ ]:
def next_states_phase_three(state, player):
    remaining, board = state
    
    #mills = find_mills(board, player) # Schritt 1
    
    # Schritt 2
    pieces = player_pieces(board, player)
    empty = empty_positions(board)
    boards_after_placing = []
    for ring_start, cell_start in pieces:
        for ring_goal, cell_goal in empty:
            new_board = copy.deepcopy(board)
            new_board[ring_start][cell_start] = 0    
            new_board[ring_goal][cell_goal] = player        
            boards_after_placing.append(new_board)
    
    boards = generate_final_boards(boards_after_placing, player) # Schritt 3
    
    return [[remaining, new_board] for new_board in boards] # Schritt 4

`next_states()` nimmt einen Zustand und einen Spieler, berechnet die aktuelle Spielphase und ruft die entsprechende Funktion zur Berechnung der nächsten Zustände, wie sie zuvor definiert wurden, auf.

In [ ]:
def next_states(state, player):
    phase =  get_player_phase(state, player)
    if phase == 1:
        return next_states_phase_one(state, player)
    elif phase == 2:
        return next_states_phase_two(state, player)
    else:
        return next_states_phase_three(state, player)

## Spielende

Die Funktion `finished()` prüft, ob das Spiel für den übergebenen Zustand zu Ende ist, wenn der der übergebene Spieler an der Reihe ist. 

Solange ein Spieler noch Steine setzen kann, ist das Spiel noch nicht entschieden. Ansonsten gibt es hier zwei Möglichkeiten, bei denen das Spiel zu Ende ist. Die erste Situation tritt ein, wenn ein Spieler weniger als drei Steine hat. Er hat dann verloren. Bei der zweiten Option verliert ein Spieler, wenn er sich in Phase 2 befindet, aber nicht mehr ziehen kann. Dies kommt vor, wenn alle benachbarten Felder seiner Steine vom Gegenspieler besetzt sind. Prüfen lässt sich dies auch, in dem man berechnet, ob keine validen Folgezustände existieren. Tritt keine der beiden Situationen ein, ist das Spiel laut dieser Funktion noch nicht beendet. Praktisch existieren noch weitere Situationen, in denen ein Spiel beendet ist. Dies ist zum Beispiel der Fall, wenn über zu viele Züge hinweg keine Mühle geschlossen wurde. Dies wird jedoch separat im Notebook `Muehle_GUI.ipynb` ausgewertet.


In [ ]:
def finished(state, current_player):
    [remaining, board] = state
    if any(number_player_pieces > 0 for number_player_pieces in remaining): return False
    if any(count_player_pieces(board, player) < 3 for player in [1, 2]): return True
    if len(next_states(state, current_player)) == 0: return True
    return False

Die Funktion `utility()` nimmt einen Zustand für den das Spiel beendet ist, also `finished(state, current_player)==True` und einen Spieler und gibt einen numerischen Wert zurück, der folgendermaßen zu interpretieren ist:
* -1 = Der Spieler hat verloren
* 0 =  Unentschieden
* 1 =  Der Spieler hat gewonnen

Da die Funktion nur mit einem Endzustand aufgerufen wird, muss die Spielerphase nicht beachtet werden. Denn es ist nur möglich in der Zug- und in der Springphase zu gewinnen. Für diese gelten alle unten genannten Regeln.

Falls der Spieler sich nicht in Phase 3 befindet, aber weniger als drei Steine auf dem Brett hat, wird -1 zurückgegeben, falls der Gegner weniger als drei Steine hat wird 1 zurückgegeben. Danach befindet man sich in einer der folgenden Situationen:

* utility = -1, der Spieler hat weniger als 3 Steine $\rightarrow$ Ergebnis zurückgegeben
* utility = 0, beide Spieler haben mehr als 3 Steine $\rightarrow$ Prüfen, ob Spieler noch ziehen kann
* utility = 1, der Gegner hat weniger als 3 Steine $\rightarrow$ Ergebnis zurückgeben

Als nächstes wird überprüft, ob der aktuelle Spieler noch ziehen kann, also ein möglicher Folgezustand existiert. Wenn nicht, wird -1 zurückgegeben, da der Spieler verloren hat.

* utility = -1, der Spieler kann nicht mehr ziehen $\rightarrow$ Spieler hat verloren

Ansonsten wird theoretisch 0 zurückgegeben, was bedeutet, dass das Spiel unentschieden ist. Allerdings wird dies praktisch nicht passieren, da `utility()` nur mit einem Zustand aufgerufen wird, der ein sicherer Endzustand ist, also entweder der Spieler oder der Gegner gewonnen hat. Ein Unentschieden existiert zwar, tritt aber auf, wenn beispielsweise die maximale Anzahl an Zügen ohne eine neue Mühle erreicht wurde. Dies wird jedoch nicht durch `utility()` überprüft.

In [ ]:
def utility(terminal_state, player):
    if count_player_pieces(terminal_state[1], player)           < 3: return -1
    if count_player_pieces(terminal_state[1], opponent(player)) < 3: return  1
    
    if len(next_states(terminal_state, player)) == 0: return -1
    return 0